In [2]:
# Pip imports
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import stan

try:
    import arviz as az
except ImportError as err:
    print("Please, install arviz for easy visualization of Stan models.")

import nest_asyncio
nest_asyncio.apply()

import pandas as pd

/home/krupkaa/anaconda3/envs/cognitive/lib/python3.12/site-packages/numpy/_core/getlimits.py:551: UserWarning: Signature b'\x00\xd0\xcc\xcc\xcc\xcc\xcc\xcc\xfb\xbf\x00\x00\x00\x00\x00\x00' for <class 'numpy.longdouble'> does not match any known type: falling back to type probe function.
This warnings indicates broken support for the dtype!
  machar = _get_machar(dtype)


In [198]:
stan_code = """

data {
    int<lower=1> N;
    matrix[N,3] x;
    vector[N] y;
}

parameters {
    real<lower=0> sigma;
    real alpha;
    array[3] int beta;
}

model {
    sigma ~ inv_gamma(0.3, 0.6);
    alpha ~ normal(0, 5);
    beta ~ identity_matrix(3)*normal(0,1);

    for (n in 1:N) {
        y[n] ~ normal(alpha + beta * x[n], sigma);
    }
}

"""

In [199]:
# Read in the file
insurance_data = pd.read_csv("insurance.csv")

key_param_df = insurance_data[['age', 'bmi', 'children']].astype(np.float64)
print (key_param_df)

       age     bmi  children
0     19.0  27.900       0.0
1     18.0  33.770       1.0
2     28.0  33.000       3.0
3     33.0  22.705       0.0
4     32.0  28.880       0.0
...    ...     ...       ...
1333  50.0  30.970       3.0
1334  18.0  31.920       0.0
1335  18.0  36.850       0.0
1336  21.0  25.800       0.0
1337  61.0  29.070       0.0

[1338 rows x 3 columns]


In [200]:
data_dict = {
    'N': insurance_data.shape[0],
    #'x': np.ndarray(key_param_df.values, dtype=float)
    'x': key_param_df.values,
    'y': insurance_data['charges'].values
}

model = stan.build(stan_code, data=data_dict)

Building...


Building: Syntax error:   -------------------------------------------------
    16:      sigma ~ inv_gamma(0.3, 0.6);
    17:      alpha ~ normal(0, 5);
    18:      beta ~ identity_matrix(3)*normal(0,1);
                                      ^
    19:  
    20:      for (n in 1:N) {
   -------------------------------------------------

Ill-formed "~"-statement. Expect either ";" or a truncation with the format "T[" optional expression "," optional expression "];".

ValueError: Syntax error

In [201]:
fit = model.sample(num_chains=4, num_samples=100, num_warmup=50)

Sampling:   0%
Sampling:   0% (1/600)
Sampling:   0% (2/600)
Sampling:   0% (3/600)
Sampling:   1% (4/600)
Sampling:   9% (54/600)
Sampling:  17% (104/600)
Sampling:  26% (154/600)
Sampling:  34% (204/600)
Sampling:  50% (303/600)
Sampling:  67% (402/600)
Sampling:  84% (501/600)
Sampling: 100% (600/600)
Sampling: 100% (600/600), done.
Messages received during sampling:
  Gradient evaluation took 0.000325 seconds
  1000 transitions using 10 leapfrog steps per transition would take 3.25 seconds.
  Adjust your expectations accordingly!
           three stages of adaptation as currently configured.
           Reducing each adaptation stage to 15%/75%/10% of
           the given number of warmup iterations:
             init_buffer = 7
             adapt_window = 38
             term_buffer = 5
  Gradient evaluation took 0.000322 seconds
  1000 transitions using 10 leapfrog steps per transition would take 3.22 seconds.
  Adjust your expectations accordingly!
           three stages of adap

In [124]:
az.summary(fit)

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
sigma,17877.079,189.692,17550.324,18275.394,10.411,12.973,330.0,259.0,1.03
alpha,4.925,4.889,-4.097,13.215,0.669,0.296,56.0,93.0,1.08
beta,4.023,0.925,2.282,5.620,0.063,0.048,216.0,222.0,1.01
